In [1]:
import pandas as pd
import numpy as np
import json
import matplotlib.pylab as plt
%matplotlib inline

from tyssue.core.sheet import Sheet

from tyssue import config
from tyssue.topology.sheet_topology import cell_division

from tyssue.geometry.sheet_geometry import SheetGeometry as geom
from tyssue.solvers.sheet_vertex_solver import Solver as solver
from tyssue.dynamics.sheet_vertex_model import SheetModel as model

from tyssue.core.generation import extrude
from tyssue.core.monolayer import Monolayer, MonolayerWithLamina
from tyssue.geometry.bulk_geometry import BulkGeometry, MonoLayerGeometry
from tyssue.dynamics.bulk_model import BulkModel, LaminaModel, set_model
from tyssue.solvers.sheet_vertex_solver import Solver

from tyssue.draw.vispy_draw import vp_view

from tyssue.draw.plt_draw import sheet_view

from tyssue.topology.sheet_topology import remove_face
from tyssue.core.objects import get_opposite
from tyssue.io.hdf5 import save_datasets, load_datasets

from tyssue.topology.monolayer_topology import cell_division

In [2]:
datasets = load_datasets('small_ellipsoid.hf5',
                         data_names=['vert', 'edge',
                                     'face', 'cell'])
specs = config.geometry.bulk_spec()

monolayer = Monolayer('ell', datasets, specs)
monolayer.reset_index()
monolayer.reset_topo()


# ## Settings for the quasi-static model
# ### Apical layer
apical_spec = {
    'edge': {
        'line_tension': 0.12,},
    'face': {
        'contractility': 0.04,
        'prefered_area': 0.8,
        'area_elasticity': 1.0},
    'cell': {
        'prefered_vol': 1.0,
        'vol_elasticity': 1.0},
    'settings': {
        'grad_norm_factor': 1.0,
        'nrj_norm_factor': 1.0},}

# ### Modifier for each segment of the stratum
modifiers = {
    'sagittal': {
        'edge': {
            'line_tension': 0.1},
        'face': {
            'contractility': 0.05,
            'prefered_area': 0.2}
    },
    'basal': {
        'edge': {
            'line_tension': 0.5},
        'face': {
            'contractility': 0.1,
            'prefered_area': 0.2}},
    }

set_model(monolayer, BulkModel, apical_spec, modifiers)





In [3]:
mother = 10


In [4]:
mother_faces = set(
    monolayer.edge_df[monolayer.edge_df['cell'] == mother]['face'])

monolayer.face_df['visible'] = 0
monolayer.face_df.loc[mother_faces, 'visible'] = 1

#res = Solver.find_energy_min(monolayer, BulkGeometry, BulkModel)
draw_specs = config.draw.sheet_spec()
draw_specs['face']['visible'] = True
canvas, view = vp_view(monolayer, **draw_specs)

/home/guillaume/anaconda3/envs/py35/lib/python3.5/site-packages/IPython/config.py:13: ShimWarning: The `IPython.config` package has been deprecated. You should import from traitlets.config instead.
  "You should import from traitlets.config instead.", ShimWarning)


In [5]:

daughter = cell_division(monolayer, mother, orientation='vertical')



In [9]:
monolayer.face_df['visible'] = 0

mother_faces = set(
    monolayer.edge_df[monolayer.edge_df['cell'] == mother]['face'])
daughter_faces = set(
    monolayer.edge_df[monolayer.edge_df['cell'] == daughter]['face'])
monolayer.face_df.loc[daughter_faces, 'visible'] = 1
monolayer.face_df.loc[mother_faces, 'visible'] = 0

In [10]:

colors = plt.cm.viridis(monolayer.edge_df['cell'] /
                        monolayer.edge_df['cell'].max())
draw_specs['edge']['color'] = colors

canvas, view = vp_view(monolayer, **draw_specs)

In [8]:
res = solver.find_energy_min(monolayer, MonoLayerGeometry, BulkModel)
